# Vector search in Python (Azure AI Search)
このサンプルノートブックでは Azure AI Search の Python SDK を使用して以下の検索クエリーを試すことができます。

- キーワード検索
- ベクトル検索
- ハイブリッド検索
- セマンティックハイブリッド検索



# 事前準備
この Python サンプルを実行するには、以下が必要です：
- Azure AI Search リソース。エンドポイントと管理 API キーが必要です。
- Azure OpenAI Service にアクセスできる承認済み Azure サブスクリプション
- Azure OpenAI Service への `text-embedding-ada-002` Embeddings モデルのデプロイメント。このデモでは、API バージョン `2023-05-15` を使用しています。デプロイ名はモデルと同じ「`text-embedding-ada-002`」を使用しています。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI Embeddings モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。

## パッケージのインストール

In [ ]:
!pip install azure-search-documents==11.4.0
!pip install openai

## 必要なライブラリと環境変数のインポート

In [ ]:
import azure.search.documents
azure.search.documents.__version__

In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

## 接続設定

In [ ]:
service_endpoint: str = "<Your search service endpoint>"
service_query_key: str = "<Your search service query key>"
index_name: str = "gptkbindex"
model: str = "text-embedding-ada-002"

credential = AzureKeyCredential(service_query_key)

# 1. キーワード検索
最もシンプルなキーワード検索のクエリーです。`ja.lucene` というスタンダードな日本語アナライザーに搭載されている辞書ベースのトークナイザーによって、これらのトークンに分解されます。このトークンを用いて転置インデックスが構築されます。

次に TF/IDF ベースの [BM25](https://ja.wikipedia.org/wiki/Okapi_BM25) スコアリングアルゴリズムによって、文章中からトークンの一致頻度を見て関連性スコアが決定されます。細かくはトークンのレア度や文章中の密度なども重みづけされます。キーワード検索では、わざとスペルミスしたので、**源実朝**という一つのトークンにならずに、**一文字ずつのトークンになって関係ない人名の部分でヒット**してしまったりしてますね。また、「特徴」というワードをそのまま検索しており、文章中に「和歌の特徴」とあからさまに書いていない限りこのワードは意味を成しません。残念ながらこの検索方法では、「和歌の特徴を知りたい」というユーザーの意図は考慮されません。

In [ ]:
query = "源実友のお歌にはどのような特徴があったのでしょうか？"  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
docs = search_client.search(
    search_text=query,
    top=3,
    highlight_fields="content-3",
    select="sourcepage,content,category"
) 

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

# 2. ベクトル類似性検索

## Azure OpenAI の設定

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"

In [ ]:
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt  

client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# タイトルフィールドとコンテンツフィールドのEmbeddingsを生成する関数。
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# 2.1. シンプルなベクトル検索
「源実**友**の**お歌**にはどのような特徴があったのでしょうか？」というわざとスペルミスを入れたり和歌をお歌と言い換えたりしたクエリーで検索をかけます。`text-embeddings-ada-002` で生成したベクトルを検索すると、キーワードの一致にとらわれずに、テキストの類似性だけを見て検索しています。スペルミスも良い具合に無視されて検索されることが分かります。ほしい答えは「万葉風の歌人」という部分です。

In [ ]:
query = "源実友のお歌にはどのような特徴があったのでしょうか？"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="embedding")

docs = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
)

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

## 2.1.1. 多言語能力
`text-embeddings-ada-002` の多言語能力を確認してみましょう。日本語のドキュメントを英語で検索します。

In [ ]:
query = "What were the characteristics of Minamoto Sanetomo's poetry?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="embedding")

docs = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
)

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

# 2.2. ハイブリッド検索
ハイブリッド検索では、キーワード検索とベクトル検索の両方をクエリーとして使います。ハイブリッド検索ではどのように検索スコアを計算しているかといいますと、まずキーワード検索のスコアには Okapi BM25 アルゴリズムによるスコアを採用し、ベクトル検索ではコサイン類似度をスコアとして用いており、両者異なるスコアです。これを融合するシンプルな計算式として、[Reciprocal Rank Fusion(RRF)](https://learn.microsoft.com/azure/search/hybrid-search-ranking) が採用されています。RRF は両者の文書ランクの逆数の和を取ります。つまり、その文書の順位だけを見て、どちらのランキングでも上位にきていればスコアが高くなる仕組みです。

In [ ]:
query = "源実友は征夷大将軍として知られているだけでなく、ある有名な趣味も持っています。それは何ですか。"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    top=3
)

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

# 2.3 セマンティックハイブリッド検索
セマンティックハイブリッド検索（ハイブリッド検索＋セマンティックランカー）は Azure AI Search 独自の検索機能であり、ハイブリッド検索と検索結果を高い精度で並べ替えるリランカー機能（セマンティックランカー）を組み合わせた高度な検索方法です。リランカーは Microsoft 製の言語モデルである [Turing](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/introducing-multilingual-support-for-semantic-search-on-azure/ba-p/2385110) モデルに基づいています。

In [ ]:
query = "１３人の合議制に含まれるメンバー一覧"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = docs.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")
    
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

## 2.3.1. セマンティックアンサーとセマンティックキャプション
**セマンティックアンサー**は、検索ドキュメントから答えにもっとも近いと思われる一節を取り出して表示する特殊なレスポンスです。一節の中でもさらに回答に近いと思われる部分には自動的にハイライトを付加します。セマンティックアンサーを返すには、回答の言語特性を持つフレーズまたは文が検索ドキュメントに存在している必要があり、検索クエリ自体が質問として提示されている必要があります。モデルは利用可能なコンテンツから一連の潜在的な回答を抽出し、**十分に高い信頼水準に達するときだけ**セマンティックアンサーを提案します。

**セマンティックキャプション**はピンポイントでセマンティックアンサーが得られなった場合でも、関連性の高い検索結果の一部を取り出す（抽出的要約）ことができます。これらは新しい文やフレーズを生成しているわけではないため、信頼性が必要な説明や定義を表示するシーンに適しています。

In [ ]:
query = "源頼朝が征夷大将軍に任命されたのはいつ"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = docs.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in docs:
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

In [ ]:
query = "守護・地頭を設置した人は誰"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = docs.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in docs:
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")